<a href="https://colab.research.google.com/github/ilikemichael/ml1216/blob/main/embedding_gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 5s 0us/step


In [2]:
import os
import glob
import pandas as pd

base = os.path.dirname(dataset)
def get_data(category):
    contents, targets = [], []
    dir = os.path.join(base, "aclImdb", category, "pos")
    lfn = glob.glob(os.path.join(dir, "*.txt"))
    ufn = glob.glob(os.path.join(dir, "*.TXT"))
    for fn in lfn + ufn:
        with open(fn, "r", encoding="utf-8") as f:
            contents.append(f.read())
            targets.append(1)
    dir = os.path.join(base, "aclImdb", category, "neg")
    lfn = glob.glob(os.path.join(dir, "*.txt"))
    ufn = glob.glob(os.path.join(dir, "*.TXT"))
    for fn in lfn + ufn:
        with open(fn, "r", encoding="utf-8") as f:
            contents.append(f.read())
            targets.append(0)
    df = pd.DataFrame({
        "content":contents,
        "target":targets
    })
    return df
train_df = get_data("train")
test_df = get_data("test")

In [3]:
# tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit
tok.fit_on_texts(train_df["content"])

In [4]:
# transform
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])

In [ ]:

pd.DataFrame(x_train_seq)
tok.index_word[3005]
tok.word_index

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)

In [7]:
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,794,38,944,84,1693,2,38,30,1,34,25,54,240,712,15,38,8,4,1,462,7,7,14,59,132,10,27,3,2032,8,972,49,151,10,158,132,27,3,2547
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1076,46,82,1132,1023,30,726,209,11,19,1404,7,7,249,22,710,167,5,847,229,39,25,3,18,44,22,137,16,32,906,327,22,77,50,71,1326,25,3,49,55
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,150,1,372,104,2178,68,614,60,825,1,1117,1473,7,7,8,1171,22,77,116,11,120,258,1,83,238,2178,187,44,22,398,146,1,120,91,1379,77,2633,122,2,699
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,201,19,6,1,1727,19,2,16,42,1148,20,247,2,765,2246,124,303,1,168,545,136,9,1577,931,3,774,133,9,117,1,344,14,3,212,64,19,8,11,509
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,138,12,405,1,17,3,2095,1803,2,116,5,103,3,212,64,17,15,29,1,81,34,261,8,280,116,31,227,1,88,726,30,219,28,4,95,17,4,29,208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,314,69,35,1254,10,1056,543,699,36,55,5,55,50,14,1,19,10,516,758,3,454,155,2,31,12,11,239,1011,54,50,71,3,339,15,152,2116,634,27,467,155
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,855,11,17,63,683,69,9,6,547,725,2,193,30,208,1,62,6,21,786,569,8,1,1516,790,102,58,2011,329,1,271,42,3,2726,62,60,1978,42,1488,20,265
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,15,58,229,749,1082,454,13,287,146,7,7,2,58,319,555,124,21,57,165,49,8,11,17,237,21,58,549,35,58,124,21,381,73,7,7,10,199,9,3,297
24998,59,25,74,2378,14,3,330,33,2468,32,2651,2,33,2468,24,116,15,34,6,244,946,174,1,436,16,11,233,151,6,12,33,112,384,9,137,280,2298,5,15,1,...,66,54,1097,26,8,5,166,3,179,12,13,679,2,192,5,384,87,166,1,541,12,59,94,87,1,83,726,1099,2,36,3,530,2195,5,3,129,725,16,3,1200


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling1D
layers = [
    # 3001 * 128
    Embedding(3001, 128, mask_zero=True, input_length=512),
    GlobalAveragePooling1D(),
    Dense(2, activation="softmax")

]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [10]:

y_train = train_df["target"]
y_test = test_df["target"]

In [11]:

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("model.h5", save_best_only=True)
]
# validation_split: 切出一部分資料驗證
# batch_size: 看多少筆才做一次調整(梯度下降)
# epochs: 訓練次數(60000-6000筆/epoch)
# 1 epoch 多少次梯度下降: 54000 / 200 -> 270
# verbose: 印出多少log(1:default 0:quiet 2:)
model.fit(x_train_pad, 
     y_train,
     validation_split=0.1,
     batch_size=200,
     epochs=50,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 7s - loss: 0.6346 - accuracy: 0.6409 - val_loss: 0.6595 - val_accuracy: 0.5988
Epoch 2/50
113/113 - 6s - loss: 0.4675 - accuracy: 0.8246 - val_loss: 0.4905 - val_accuracy: 0.7708
Epoch 3/50
113/113 - 6s - loss: 0.3632 - accuracy: 0.8648 - val_loss: 0.3881 - val_accuracy: 0.8324
Epoch 4/50
113/113 - 6s - loss: 0.3128 - accuracy: 0.8808 - val_loss: 0.3226 - val_accuracy: 0.8648
Epoch 5/50
113/113 - 6s - loss: 0.2835 - accuracy: 0.8922 - val_loss: 0.3439 - val_accuracy: 0.8508
Epoch 6/50
113/113 - 6s - loss: 0.2634 - accuracy: 0.9001 - val_loss: 0.3537 - val_accuracy: 0.8468
Epoch 7/50
113/113 - 6s - loss: 0.2492 - accuracy: 0.9055 - val_loss: 0.3444 - val_accuracy: 0.8504
Epoch 8/50
113/113 - 6s - loss: 0.2379 - accuracy: 0.9092 - val_loss: 0.3291 - val_accuracy: 0.8604
Epoch 9/50
113/113 - 6s - loss: 0.2293 - accuracy: 0.9118 - val_loss: 0.3666 - val_accuracy: 0.8456


In [12]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 4s 5ms/step - loss: 0.3216 - accuracy: 0.8718


[0.3215848207473755, 0.8717600107192993]

In [13]:
#如果不帶入input_length, 不限制輸入長度, 把第一層的weights取出,然後直接寫入第一層的weights

layers = [
    Embedding(3001, 128, mask_zero=True)
]
partial = Sequential(layers)
w = model.layers[0].get_weights()
partial.layers[0].set_weights(w)
partial.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [14]:
x_train_pad

array([[   0,    0,    0, ...,   27,    3, 2547],
       [   0,    0,    0, ...,    3,   49,   55],
       [   0,    0,    0, ...,  122,    2,  699],
       ...,
       [   0,    0,    0, ...,    9,    3,  297],
       [  59,   25,   74, ...,   16,    3, 1200],
       [   0,    0,    0, ...,    5,    1, 2768]], dtype=int32)

In [15]:
#這裡展示每一張詞,都有128個weights
import random
#idx = 3000 #看最後一張
idx = random.randint(1, 3000)
print("詞:", tok.index_word[idx])
partial.predict([[idx]]) #這裡predict輸入的list輸入的東東,要跟x_train_pad一樣的shape


詞: portrayed


array([[[-2.5128366e-03, -7.3985375e-02, -2.1817358e-02, -5.8619026e-02,
          1.2692330e-02,  6.1835639e-02,  1.1139840e-02, -1.7806239e-02,
         -3.1172963e-02,  4.5766834e-02, -4.5064517e-05,  2.1705288e-02,
          4.6970383e-03, -9.5757619e-02,  3.8422355e-03,  2.0080826e-03,
          7.8372538e-02, -5.0896853e-02, -9.2065334e-03, -5.0647371e-03,
          6.3734795e-03, -5.0550385e-04, -4.7660619e-02,  4.8529413e-02,
          5.9762273e-02,  7.4120849e-02,  1.0804570e-02, -3.0530049e-02,
         -7.1934476e-02,  1.3473630e-02,  1.7772108e-02,  2.0156428e-02,
          3.4958902e-03, -2.1720467e-02, -7.3544779e-03, -1.8607967e-02,
         -6.6044323e-02, -7.1073871e-04,  7.6929338e-02,  7.4419059e-02,
         -5.0759230e-02, -4.9267404e-02, -1.8389544e-02, -1.8966358e-02,
          3.1701587e-02,  1.0492127e-02,  8.7046027e-03, -7.3201999e-02,
         -9.6846512e-03,  6.6379979e-02, -5.0189566e-02,  1.2654033e-03,
         -7.1308449e-02, -7.4916422e-02,  4.2573150